In [1]:
API_KEY= [UR_KEY]
#GEMINI_API_KEY =[UR_KEY]

In [ ]:
import os
import json
import time
import tiktoken
import numpy as np
import pandas as pd
from tqdm import tqdm
from datasets import load_dataset
from openai import OpenAI
import google.generativeai as genai
import logging
import re

In [2]:
#GEMINI ONE
import os
import json
import time
import tiktoken
import numpy as np
import pandas as pd
from tqdm import tqdm
from datasets import load_dataset
from openai import OpenAI
import google.generativeai as genai
import logging
import re


# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Gemini API Configuration
genai.configure(api_key=GEMINI_API_KEY)

# Load MATH-500 dataset
ds = load_dataset("HuggingFaceH4/MATH-500")

# Models to Compare
models = {
    "Gemini 2.0 Flash": "gemini-2.0-flash",
}

tokenizer = tiktoken.get_encoding("cl100k_base")

In [10]:
#OPEN Router One

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# OpenRouter Client
client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=API_KEY,
)

# Gemini API Configuration
genai.configure(api_key=GEMINI_API_KEY)

# Load MATH-500 dataset
ds = load_dataset("HuggingFaceH4/MATH-500")

# Models to Compare
models = {
    "LLaMA 3 8B": "meta-llama/llama-3-8b-instruct:free",
    "DeepSeek Chat": "deepseek/deepseek-chat:free",
    "Dolphin 3.0 R1": "cognitivecomputations/dolphin3.0-r1-mistral-24b:free",
}

#DO one model at time

tokenizer = tiktoken.get_encoding("cl100k_base")

In [ ]:
def create_prompt(problem, level="unknown"):
    return f"""Below is a math problem. Solve it step by step and provide the final answer.

PROBLEM:
{problem}

Please take your time to work through this carefully. Do all the steps and reason it properly and give me all the steps.
In case of a word problem, where the answer is in a sentence, give the answer in a word and remove the units like inches or cm etc.
Give me the final answer as
FINAL ANSWER: [your_answer]
Make sure this final answer is a numerical value or word  or an expression. In case of an expression or a word, return it in LaTeX format. """

def clean_latex_answer(answer):
    """
    Clean and normalize LaTeX answers by removing \boxed{}, \left, \right, and inline math markers.
    """
    # Remove all instances of \boxed{...}
    answer = re.sub(r'\\boxed\s*{\s*(.*?)\s*}', r'\1', answer)

    # Remove \left and \right
    answer = re.sub(r'\\left\s*|\s*\\right', '', answer)

    # Remove LaTeX inline math markers like \( ... \) and $...$
    answer = re.sub(r'\\\(|\\\)|\$', '', answer)

    # Remove extra spaces and strip trailing characters
    answer = answer.strip().rstrip(".,:;")
    
    return answer

def extract_answer(response):
    # Check for common final answer patterns
    final_answer_patterns = [
        "Final answer:", "FINAL ANSWER:", "final answer is", "answer:", "Answer:"
    ]

    for phrase in final_answer_patterns:
        if phrase in response:
            answer_part = response.split(phrase)[-1].strip()
            
            # Apply cleanup using regex
            answer_part = clean_latex_answer(answer_part)

            # Remove brackets
            answer_part = answer_part.strip('[]')
            decimal_match = re.search(r"[-+]?\d*\.\d+", answer_part)
            if decimal_match:
                return decimal_match.group(0)

            # Get the first line or until a period
            if "\n" in answer_part:
                return answer_part.split("\n")[0].strip().rstrip(".")
            elif "." in answer_part:
                return answer_part.split(".")[0].strip()
            else:
                return answer_part.strip()

    # For word problems (name, single word answers)
    lines = response.strip().split('\n')
    for line in reversed(lines):
        line = line.strip()
        if '"' in line or "'" in line:
            try:
                return line.split('"')[1] if '"' in line else line.split("'")[1]
            except IndexError:
                continue
        elif len(line.split()) == 1:
            return line

    return "Answer not found"




def normalize_answer(answer):
    """Normalize answer format for consistent comparison."""
    answer = str(answer).strip()

    # Convert LaTeX variants with \text{} to plain text
    if answer.startswith(r"\text{") and answer.endswith("}"):
        answer = answer[6:-1]  # Extract the content within \text{}

    # Remove LaTeX block markers
    answer = re.sub(r'\\\[|\]', '', answer)

    # Remove ** or other extra symbols
    answer = re.sub(r'^\*+|\*+$', '', answer)

    # Handle LaTeX-like outputs
    answer = answer.replace("\\dfrac", "\\frac")
    answer = answer.replace("\\left", "").replace("\\right", "")
    answer = answer.replace("^\\circ", "").replace("^{\\circ}", "").replace("^°", "")
    answer = answer.replace("}{", "/").replace("{", "").replace("}", "")

    # Handle LaTeX matrices (pmatrix, bmatrix, etc.)
    matrix_pattern = r"\\begin\{(?:bmatrix|pmatrix|vmatrix|Vmatrix)\}(.*?)\\end\{(?:bmatrix|pmatrix|vmatrix|Vmatrix)\}"
    matrix_match = re.search(matrix_pattern, answer, re.DOTALL)

    if matrix_match:
        matrix_content = matrix_match.group(1)
        # Clean up the matrix and format it as a simple vector
        matrix_vector = "[" + ", ".join(matrix_content.split("\\\\")) + "]"
        answer = matrix_vector.strip()

    return answer.strip().rstrip(".,:;")



def check_numeric_equality(val1, val2):
    try:
        # Handle fractions
        if "/" in val1 and "/" in val2:
            try:
                v1_num, v1_denom = map(float, val1.split("/"))
                v2_num, v2_denom = map(float, val2.split("/"))

                # Check for division by zero
                if v1_denom == 0 or v2_denom == 0:
                    logger.warning(f"Division by zero detected when comparing {val1} and {val2}")
                    return False

                return abs((v1_num / v1_denom) - (v2_num / v2_denom)) < 1e-6
            except ValueError as e:
                logger.warning(f"Error parsing fractions {val1} and {val2}: {str(e)}")
                return False

        # Handle direct numeric comparison
        try:
            return abs(float(val1) - float(val2)) < 1e-6
        except ValueError as e:
            logger.warning(f"Error converting to float {val1} and {val2}: {str(e)}")
            return False
    except Exception as e:
        logger.error(f"Unexpected error in numeric equality check between {val1} and {val2}: {str(e)}")
        return False

def check_angular_equality(val1, val2):
    """Compare angular values, handling degrees and radians."""
    try:
        # Convert all to radians for comparison
        def to_radians(val):
            val = val.replace("\\pi", "pi").replace("π", "pi")

            # Handle degrees
            if "°" in val:
                try:
                    degrees = float(val.replace("°", ""))
                    return degrees * (3.14159265359 / 180)
                except ValueError:
                    pass

            # Handle pi fractions (pi/2, 3pi/4, etc.)
            if "pi" in val:
                try:
                    if "/" in val:
                        parts = val.replace("pi", "").split("/")
                        if parts[0].strip() == "":
                            parts[0] = "1"
                        num = float(parts[0])
                        denom = float(parts[1])
                        return num * 3.14159265359 / denom
                    else:
                        multiplier = 1.0
                        if val.replace("pi", "").strip():
                            multiplier = float(val.replace("pi", ""))
                        return multiplier * 3.14159265359
                except (ValueError, ZeroDivisionError):
                    pass

            # If it's just a number, return it
            try:
                return float(val)
            except ValueError:
                return val

        # Convert both values to radians
        val1_rad = to_radians(val1)
        val2_rad = to_radians(val2)

        # If conversion failed (returned non-numeric), do string comparison
        if isinstance(val1_rad, str) or isinstance(val2_rad, str):
            return val1 == val2

        # Compare angles in radians, taking periodicity into account
        val1_mod = val1_rad % (2 * 3.14159265359)
        val2_mod = val2_rad % (2 * 3.14159265359)

        return abs(val1_mod - val2_mod) < 1e-4
    except Exception as e:
        logger.error(f"Error in angular equality check: {str(e)}")
        return False

def check_answer(predicted, correct):
    """Check if predicted answer matches correct answer with improved normalization."""
    try:
        # Normalize both answers
        predicted_norm = normalize_answer(predicted)
        correct_norm = normalize_answer(correct)

        # Direct string comparison after normalization
        if predicted_norm == correct_norm:
            return True

        # Strip all LaTeX and spaces for strict comparison
        predicted_clean = predicted_norm.replace("\\", "").replace(" ", "").lower()
        correct_clean = correct_norm.replace("\\", "").replace(" ", "").lower()

        if predicted_clean == correct_clean:
            return True

        # Special handling for common cases

        # 1. Angular measurements
        if "°" in predicted_norm or "°" in correct_norm or "pi" in predicted_norm or "pi" in correct_norm:
            return check_angular_equality(predicted_norm, correct_norm)

        # 2. Numeric equality for numbers and fractions
        try:
            # Try direct numeric comparison for simple numbers
            if predicted_clean.replace(".", "").isdigit() and correct_clean.replace(".", "").isdigit():
                return abs(float(predicted_clean) - float(correct_clean)) < 1e-6

            # Handle fractions
            if "/" in predicted_clean and "/" in correct_clean:
                p_parts = predicted_clean.split("/")
                c_parts = correct_clean.split("/")

                if len(p_parts) == 2 and len(c_parts) == 2:
                    p_num, p_denom = float(p_parts[0]), float(p_parts[1])
                    c_num, c_denom = float(c_parts[0]), float(c_parts[1])

                    # Check for division by zero
                    if p_denom == 0 or c_denom == 0:
                        return False

                    return abs((p_num / p_denom) - (c_num / c_denom)) < 1e-6
        except ValueError:
            pass

        # 3. Coordinate pairs (x,y) or (r,θ)
        if "(" in predicted_clean and ")" in predicted_clean and "(" in correct_clean and ")" in correct_clean:
            try:
                # Extract values inside parentheses
                p_coords = predicted_clean.split("(")[1].split(")")[0].split(",")
                c_coords = correct_clean.split("(")[1].split(")")[0].split(",")

                if len(p_coords) == 2 and len(c_coords) == 2:
                    # Compare first coordinate
                    if p_coords[0] != c_coords[0]:
                        try:
                            if abs(float(p_coords[0]) - float(c_coords[0])) > 1e-6:
                                return False
                        except ValueError:
                            if p_coords[0] != c_coords[0]:
                                return False

                    # Compare second coordinate (could be angle)
                    if "pi" in p_coords[1] or "pi" in c_coords[1]:
                        return check_angular_equality(p_coords[1], c_coords[1])
                    else:
                        try:
                            return abs(float(p_coords[1]) - float(c_coords[1])) < 1e-6
                        except ValueError:
                            return p_coords[1] == c_coords[1]
            except Exception as e:
                logger.warning(f"Error comparing coordinates: {str(e)}")

        # If all else fails, return False
        return False
    except Exception as e:
        logger.error(f"Error comparing answers '{predicted}' and '{correct}': {str(e)}")
        return False

import time

request_count = 0

def query_model(model_id, prompt):
    global request_count
    try:
        if model_id == "gemini-2.0-flash":
            model = genai.GenerativeModel(model_id)
            response = model.generate_content(prompt)
            result = response.text
        else:
            completion = client.chat.completions.create(
                model=model_id,
                messages=[{"role": "user", "content": prompt}],
                temperature=0.1,
                max_tokens=2048
            )
            if not completion or not completion.choices:
                return "Error: Empty response from model."
            result = completion.choices[0].message.content

        request_count += 1

        # After every 15 requests, wait for 30 seconds
        if request_count % 15 == 0:
            print("Rate limit reached. Pausing for 30 seconds...")
            time.sleep(30)

        return result
        
    except Exception as e:
        logger.error(f"Error querying model {model_id}: {str(e)}")
        return f"Error: {str(e)}"


def evaluate_models():
    results = {model_name: {"correct": 0, "total": 0, "responses": [], "errors": []} for model_name in models}
    test_data = list(ds["test"])[:250]  #Sample size

    print(f"Evaluating {len(test_data)} samples.")

    for idx, sample in enumerate(tqdm(test_data, desc="Evaluating")):
        try:
            problem = sample['problem']
            correct_answer = sample['answer']
            level = sample.get('level', 'unknown')

            prompt = create_prompt(problem, level)

            for model_name, model_id in models.items():
                try:
                    response = query_model(model_id, prompt)

                    try:
                        predicted_answer = extract_answer(response)
                    except Exception as e:
                        logger.error(f"Failed to extract answer for problem {idx}: {str(e)}")
                        predicted_answer = "Error extracting answer"
                        results[model_name]['errors'].append({
                            "problem_idx": idx,
                            "error_type": "extraction_error",
                            "error_msg": str(e)
                        })

                    try:
                        is_correct = check_answer(predicted_answer, correct_answer)
                    except Exception as e:
                        logger.error(f"Failed to check answer for problem {idx}: {str(e)}")
                        is_correct = False
                        results[model_name]['errors'].append({
                            "problem_idx": idx,
                            "error_type": "check_error",
                            "error_msg": str(e)
                        })

                    results[model_name]['total'] += 1
                    if is_correct:
                        results[model_name]['correct'] += 1

                    results[model_name]['responses'].append({
                        "problem": problem,
                        "level": level,
                        "correct_answer": correct_answer,
                        "model_response": response,
                        "extracted_answer": predicted_answer,
                        "is_correct": is_correct
                    })
                except Exception as e:
                    logger.error(f"Failed to process sample {idx} with model {model_name}: {str(e)}")
                    results[model_name]['errors'].append({
                        "problem_idx": idx,
                        "error_type": "processing_error",
                        "error_msg": str(e)
                    })
        except Exception as e:
            logger.error(f"Failed to process sample {idx}: {str(e)}")
            for model_name in models:
                results[model_name]['errors'].append({
                    "problem_idx": idx,
                    "error_type": "sample_error",
                    "error_msg": str(e)
                })

    return results

def calculate_metrics(results):
    metrics = {}
    for model_name, data in results.items():
        accuracy = (data['correct'] / data['total'] * 100) if data['total'] > 0 else 0
        error_count = len(data['errors'])
        metrics[model_name] = {
            "Overall Accuracy": f"{accuracy:.2f}%",
            "Error Count": error_count,
            "Completed Samples": data['total'],
            "Errors Per Sample": f"{error_count/data['total']:.4f}" if data['total'] > 0 else "N/A"
        }
    return metrics

def print_answer_comparison(results):
    """Print a side-by-side comparison of extracted answers vs correct answers with normalization."""
    print("\n===== ANSWER COMPARISON =====")
    print(f"{'Problem #':<8} {'Model':<15} {'Correct?':<8} {'Raw Extracted':<30} {'Normalized':<25} {'Correct Answer':<30}")
    print("-" * 120)

    for model_name, data in results.items():
        for idx, response_data in enumerate(data['responses']):
            correct_mark = "✓" if response_data['is_correct'] else "✗"
            extracted = response_data['extracted_answer']
            correct = response_data['correct_answer']

            # Show normalized version too
            normalized = normalize_answer(extracted)

            # Truncate long answers for display
            if len(str(extracted)) > 28:
                extracted = str(extracted)[:25] + "..."
            if len(str(normalized)) > 23:
                normalized = str(normalized)[:20] + "..."
            if len(str(correct)) > 28:
                correct = str(correct)[:25] + "..."

            print(f"{idx:<8} {model_name:<15} {correct_mark:<8} {extracted:<30} {normalized:<25} {correct:<30}")

    print("\n")

def run_evaluation():
    print("Starting MATH-500 model evaluation...")
    results = evaluate_models()
    metrics = calculate_metrics(results)

    print("\n===== EVALUATION RESULTS =====")
    for model_name, data in metrics.items():
        print(f"{model_name}: {data['Overall Accuracy']} (Errors: {data['Error Count']})")

    # Print answer comparison
    print_answer_comparison(results)

    # Save detailed results to file
    os.makedirs("results", exist_ok=True)
    with open("results/math500_detailed_results.json", "w") as f:
        json.dump(results, f, indent=2)
    with open("results/math500_metrics.json", "w") as f:
        json.dump(metrics, f, indent=2)

    # Create a separate answer comparison file
    answer_comparison = []
    for model_name, data in results.items():
        for idx, response_data in enumerate(data['responses']):
            answer_comparison.append({
                "problem_idx": idx,
                "problem": response_data['problem'][:100] + "..." if len(response_data['problem']) > 100 else response_data['problem'],
                "model": model_name,
                "extracted_answer": response_data['extracted_answer'],
                "correct_answer": response_data['correct_answer'],
                "is_correct": response_data['is_correct']
            })

    with open("results/math500_answer_comparison.json", "w") as f:
        json.dump(answer_comparison, f, indent=2)

    # Create a more readable CSV version
    comparison_df = pd.DataFrame(answer_comparison)
    comparison_df.to_csv("results/math500_answer_comparison.csv", index=False)

    print("Results saved to 'results/' directory.")

    # Create error report
    error_summary = {}
    for model_name, data in results.items():
        error_types = {}
        for error in data['errors']:
            error_type = error['error_type']
            if error_type not in error_types:
                error_types[error_type] = 0
            error_types[error_type] += 1
        error_summary[model_name] = error_types

    with open("results/math500_error_summary.json", "w") as f:
        json.dump(error_summary, f, indent=2)

    print("\n===== ERROR SUMMARY =====")
    for model_name, errors in error_summary.items():
        print(f"{model_name} errors:")
        for error_type, count in errors.items():
            print(f"  - {error_type}: {count}")

    comparison_df = pd.DataFrame.from_dict({k: {m: v[m] for m in ["Overall Accuracy", "Error Count"]}
                                           for k, v in metrics.items()}, orient='index').reset_index()
    comparison_df.columns = ['Model', 'Overall Accuracy (%)', 'Error Count']
    comparison_df.to_csv("results/math500_comparison.csv", index=False)
    print(comparison_df)

# Run the evaluation
run_evaluation()